In [90]:
import numpy as np



In [25]:

!unzip./ Data / orange_small_test.data.zip
!unzip./ Data / orange_small_train.data.zip

Archive:  ./Data/orange_small_test.data.zip
  inflating: orange_small_test.data  
 extracting: orange_small_test.data.cksum  
Archive:  ./Data/orange_small_train.data.zip
  inflating: orange_small_train.data  
 extracting: orange_small_train.data.cksum  


In [147]:
import pandas as pd

In [158]:
X_train = pd.read_csv('./Data/orange_small_train.data', sep='\t')
X_test = pd.read_csv('./Data/orange_small_test.data', sep='\t')
y_train_churn = pd.read_csv('./Data/orange_small_train_churn.labels.txt',header=None)
y_train_apt = pd.read_csv('./Data/orange_small_train_appetency.labels',header=None)
y_train_upsell = pd.read_csv('./Data/orange_small_train_upselling.labels',header=None)

In [159]:
X_train.shape, X_test.shape

((50000, 230), (50000, 230))

In [160]:
y_train_apt.shape, y_train_churn.shape, y_train_upsell.shape

((50000, 1), (50000, 1), (50000, 1))

In [161]:
y_train_apt.value_counts(normalize=True), y_train_churn.value_counts(normalize=True), y_train_upsell.value_counts(
    normalize=True)

(-1    0.9822
  1    0.0178
 dtype: float64,
 -1    0.92656
  1    0.07344
 dtype: float64,
 -1    0.92636
  1    0.07364
 dtype: float64)

In [165]:
# dropping any column with missing value

# collect minimum no of records to be present value for data
# using data with high percentage of missing records makes bad training set and will induce a high variance in the model predictions
missing_perc = 20
min_count = int(((100 - missing_perc) / 100) * X_train.shape[0] + 1)
min_count

40001

### Feature engineering

In [166]:
#drop all missing and only continous variable
prefered_uniques = 10
# create a list of numeric values and less than prefered uniques categorical columns
fe_unique_columns = list(X_train.select_dtypes(include='number').columns) + list(
    (X_train.select_dtypes(include='object').nunique() < prefered_uniques).index[
        X_train.select_dtypes(include='object').nunique() < prefered_uniques])
# collect only numeric features
X_train_all_missing_drop_continous = X_train.dropna( axis=1,thresh=min_count).dropna(axis=0).select_dtypes(include=['number'])#.agg(['nunique','count'])
numeric_col_after_drop = X_train_all_missing_drop_continous.columns
# create X_train with categorical variables with less than 10 uniques
X_train_all_missing_dropped_prefered_unique_cat = X_train[fe_unique_columns].dropna( axis=1,thresh=min_count).dropna(axis=0)
feature_prefered_columns = list(X_train_all_missing_dropped_prefered_unique_cat.columns)
# creating dummy variables for the data with  categorical variables
X_train_encoded = pd.get_dummies(X_train_all_missing_dropped_prefered_unique_cat,drop_first=True)
X_train_numeric = X_train_all_missing_drop_continous
del X_train_all_missing_drop_continous
# selecting X_test with the appropriate predictors for numeric predictors and selected unique threshold for category inclusion in predictors
X_test_numeric = X_test[numeric_col_after_drop]
X_test_encoded = pd.get_dummies(X_test[feature_prefered_columns],drop_first=True)[X_train_encoded.columns]


In [167]:
X_train_encoded.shape, X_test_encoded.shape#.columns

((36827, 68), (50000, 68))

In [168]:
X_test_encoded.shape

(50000, 68)

In [132]:
set(X_train_all_missing_drop_continous.columns) - set()

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var134,Var140,Var143,Var144,Var149,Var153,Var160,Var163,Var173,Var181
0,1526.0,7.0,184.0,464.0,580.0,14.0,128.0,166.56,0.0,3570.0,...,69134.0,185.0,0.0,9.0,397579.0,1812252.0,142.0,38418.0,0.0,0.0
2,5236.0,7.0,904.0,1212.0,1515.0,26.0,816.0,220.08,0.0,5883894.0,...,248932.0,800.0,0.0,36.0,320565.0,9826360.0,206.0,434946.0,0.0,0.0
4,1029.0,7.0,3216.0,64.0,80.0,4.0,64.0,200.00,0.0,0.0,...,66046.0,3255.0,0.0,9.0,267162.0,644836.0,2.0,0.0,0.0,0.0
5,658.0,7.0,3156.0,224.0,280.0,2.0,72.0,200.00,5.0,0.0,...,43684.0,355.0,0.0,9.0,2639.0,745620.0,68.0,61368.0,0.0,0.0
6,1680.0,7.0,2952.0,308.0,385.0,4.0,128.0,176.56,0.0,13158.0,...,104978.0,1665.0,0.0,18.0,47019.0,2345424.0,52.0,417438.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,266.0,0.0,0.0,792.0,990.0,18.0,432.0,336.56,0.0,2214786.0,...,227884.0,0.0,0.0,0.0,26159.0,3640968.0,104.0,138432.0,0.0,0.0
49995,357.0,0.0,0.0,132.0,165.0,2.0,0.0,288.08,0.0,6042420.0,...,0.0,0.0,0.0,9.0,604800.0,10367040.0,22.0,0.0,0.0,0.0
49996,1078.0,0.0,2736.0,380.0,475.0,2.0,88.0,166.56,0.0,0.0,...,39652.0,0.0,0.0,9.0,85120.0,1344900.0,150.0,9540.0,0.0,0.0
49997,2807.0,7.0,1460.0,568.0,710.0,4.0,328.0,166.56,0.0,42210.0,...,131588.0,675.0,0.0,27.0,4410.0,1242044.0,124.0,61734.0,0.0,0.0


In [127]:
(X_train.select_dtypes(include='object').nunique() < prefered_uniques)

Var191     True
Var192    False
Var193    False
Var194     True
Var195    False
Var196     True
Var197    False
Var198    False
Var199    False
Var200    False
Var201     True
Var202    False
Var203     True
Var204    False
Var205     True
Var206    False
Var207    False
Var208     True
Var210     True
Var211     True
Var212    False
Var213     True
Var214    False
Var215     True
Var216    False
Var217    False
Var218     True
Var219    False
Var220    False
Var221     True
Var222    False
Var223     True
Var224     True
Var225     True
Var226    False
Var227     True
Var228    False
Var229     True
dtype: bool

In [74]:
X_train.loc[:, X_train.select_dtypes(include='object').columns.nunique() < prefered_uniques]

KeyError: 'False: boolean label can not be used without a boolean index'

In [59]:
 X_train.dropna(axis=1, thresh=min_count).dropna(axis=0).select_dtypes(include='object').agg(
    ['nunique']) < prefered_uniques

,Var192,Var193,Var195,Var196,Var197,Var198,Var199,Var202,Var203,Var204,...,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var226,Var227,Var228
nunique,False,False,False,True,False,False,False,False,True,False,...,False,True,False,False,True,False,True,False,True,False


In [50]:
X_train.dropna(axis=1, thresh=min_count).dropna(axis=0)['Var196'].unique()

array(['1K8T', 'z3mO', 'mKeq', 'JA1C'], dtype=object)